# 6.1  데이터 수집 1 – 한 페이지 크롤링

### 6.1.2 다나와 검색 페이지 접속

In [ ]:
# 예제 6-1 selenium으로 다나와 검색 결과 URL에 접속
from selenium import webdriver
driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
url = "http://search.danawa.com/dsearch.php?query=무선청소기&tab=main"
driver.get(url)

### 6.1.3 다나와 검색 웹 페이지에서 상품 정보 가져오기

In [ ]:
from bs4 import BeautifulSoup

#예제 6-2 웹 페이지의 HTML 정보 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 예제 6-3 1페이지에 대한 무선청소기 정보 가져오기 1
prod_items = soup.select('li.prod_item')
len(prod_items)

In [ ]:
# 예제 6-4 1페이지에 대한 무선청소기 정보 가져오기 2
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
# 예제 6-5 1페이지에 대한 무선청소기 정보 가져오기 3
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
# 예제 6-6 상품명 정보 가져오기
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

In [ ]:
# 예제 6-7 스펙 목록 정보 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
print(spec_list)

In [ ]:
# 예제 6-8 가격 정보 가져오기
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

In [ ]:
# 예제 6-9 반복문으로 검색 결과의 1페이지에 대한 상품 정보 추출
prod_data = []
for prod_item in prod_items:
    try: # ① 상품명 가져오기
        title = prod_item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try: # ② 스펙 목록 가져오기
        spec_list = prod_item.select('div.spec_list')[0].text.strip()
    except:
        spec_list = ''
    try: # ③ 가격 정보 가져오기 
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
    except:
        price = 0
    prod_data.append([title, spec_list, price])
print(len(prod_data))
print(prod_data)

In [ ]:
# 예제 6-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
         # ① 상품명 가져오기
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
        # ② 스펙 목록 가져오기
        try: 
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
        except:
            spec_list = ''
        # ③ 가격 정보 가져오기 
        try: 
            price = int(prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",",""))
        except:
            price = 0
        prod_data.append([title, spec_list, price])
    return prod_data

In [ ]:
# 예제 6-11 상품 정보를 가져오는 함수 테스트
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

# 6.2  데이터 수집 2 - 여러 페이지에 걸친 다나와 검색 페이지 크롤링

### 6.2.1 다나와 검색 결과 페이지 URL 분석

In [ ]:
# 예제 6-12 다나와 검색 URL을 만들어주는 함수 
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)


### 6.2.2 주피터 노트북의 진행표시줄 처리 

In [ ]:
# 예제 6-13 tqdm 라이브러리 설치 
! pip install tqdm

In [ ]:
# 예제 6-14 tqdm 사용법 
import time
from tqdm import tqdm_notebook
total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. 
    time.sleep(5)

### 6.2.3 여러 페이지에 걸친 상품 정보 수집

In [ ]:
# 예제 6-15 실전 다나와 크롤링
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')

# 암묵적으로 웹 자원 로드를 위해 3초까지 기다림
driver.implicitly_wait(3)
keyword = '무선청소기'
total_page = 10
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용 
for page in tqdm_notebook(range(1, total_page + 1)):
    # ① 검색 페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    # 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)
    
    # ② 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ③ 상품 정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    # ④ 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list


### 6.2.4 수집 데이터 저장 

In [ ]:
# 예제 6-16 데이터 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('./files/3_1_danawa_crawling_result.xlsx', index = False)

### 6.3.1 다나와 크롤링 데이터 불러오기 

In [ ]:
# 예제 6-17 다나와 크롤링 결과 가져오기 
import pandas as pd

data = pd.read_excel('./files/3_1_danawa_crawling_result.xlsx')
data.info()
data.head()

### 6.3.2 회사명, 모델명 정리 

In [ ]:
# 예제 6-18 회사명 + 모델명 분리
company_list = []
product_list = []
for title in data['상품명']:
    title_info = title.split(' ', 1)
    company_name = title_info[0]
    product_name = title_info[1]
    company_list.append(company_name)
    product_list.append(product_name)

### 6.3.4 스펙 목록에서 카테고리, 사용시간, 흡입력을 추출해서 정리


In [ ]:
# 예제 6-19 첫 번째 제품의 스펙 목록 분리
spec_list = data['스펙 목록'][0].split(' / ')
spec_list 

In [ ]:
# 예제 6-20 카테고리 정보 추출
category = spec_list[0] 
category 

In [ ]:
# 예제 6-21 ‘사용시간’, ‘흡입력’이 포함된 원소 추출 
for spec in spec_list:
    if '사용시간' in spec: 
         use_time_spec = spec 
    elif '흡입력' in spec:
        suction_spec = spec  
print(use_time_spec)
print(suction_spec)

In [ ]:
# 예제 6-22 정량적인 수치 추출 
use_time_value = use_time_spec.split(' ')[1].strip() 
suction_value = suction_spec.split(' ')[1].strip()
print(use_time_value)
print(suction_value)

In [ ]:
# 예제 6-23 카테고리, 사용시간, 흡입력 추출 
category_list = []
use_time_list = []
suction_list = [] 
for spec_data in data['스펙 목록']:
    # ' / ' 기준으로 스펙 분리하기 
    spec_list = spec_data.split(' / ')
    
    # 카테고리 추출하기
    category = spec_list[0] 
    category_list.append(category)
    
    # 사용시간, 흡입력 추출 
    ## 사용시간, 흡입력 정보가 없는 제품을 위해 변수를 생성 
    use_time_value = None 
    suction_value = None 
    
    ## spec_list의 각 원소에서 사용시간, 흡입력 수치 추출
    for spec in spec_list:
        if '사용시간' in spec: 
            use_time_value = spec.split(' ')[1].strip()             
        if '흡입력' in spec:
            suction_value = spec.split(' ')[1].strip() 
    use_time_list.append(use_time_value)
    suction_list.append(suction_value)

In [ ]:
# 예제 6-24 카테고리, 사용시간, 흡입력에 대한 전처리 결과 확인
print("카테고리", len(category_list), category_list[0:5])
print("사용시간", len(use_time_list), use_time_list[0:5])
print("흡입력", len(suction_list), suction_list[0:5])